In [1]:
download("https://www.ssa.gov/oact/babynames/names.zip", "names.zip")

using ZipFile
namesZip = ZipFile.Reader("names.zip");

In [2]:
using SQLite
namesDB = SQLite.DB("names.db");

In [3]:
SQLite.execute(namesDB, "CREATE TABLE IF NOT EXISTS BabyNames (
                            year INTEGER,
                            name TEXT,
                            sex TEXT,
                            num INTEGER)");

In [ ]:
using CSV
using DataFrames

df = DataFrame()
checkNum = r"^[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)$"

for f in namesZip.files
    yearNum = SubString(f.name, 4, 7)
    if (occursin(checkNum, yearNum))
        if nrow(df) == 0
            global df = DataFrame(CSV.File(f, header=false))
            insertcols!(df, 1, :year => [String(yearNum) for i in 1:nrow(df)])
        else
            df2 = DataFrame(CSV.File(f, header=false))
            insertcols!(df2, 2, :year => [String(yearNum) for i in 1:nrow(df2)])
            append!(df, df2)
        end
    end
end

rename!(df, [:year, :name, :sex, :num])
SQLite.load!(df, namesDB, "BabyNames")

In [14]:
close(namesZip)